Cryptocurrency trading bot.

Main.

In [1]:
from crypto_logger import init_loggers
crypto_loggers = init_loggers()
crypto_loggers

{'input_15s': <cryptocurrency.crypto_logger_input.Crypto_logger_input at 0x7efcf8347310>,
 'output_15s': <cryptocurrency.crypto_logger_output.Crypto_logger_output at 0x7efc8d677730>,
 'output_1min': <cryptocurrency.crypto_logger_output.Crypto_logger_output at 0x7efcf8352400>,
 'output_30min': <cryptocurrency.crypto_logger_output.Crypto_logger_output at 0x7efc8d6a3e20>,
 'output_1h': <cryptocurrency.crypto_logger_output.Crypto_logger_output at 0x7efcf82fd0a0>,
 'output_1d': <cryptocurrency.crypto_logger_output.Crypto_logger_output at 0x7efc8d581ee0>}

In [2]:
#from crypto_logger import loop_loggers
#loop_loggers(crypto_loggers)

In [3]:
input_15s = crypto_loggers['input_15s']
df_input_15s = input_15s.dataset
df_input_15s

,base_asset,price_change_percent,bid_volume,ask_volume,bid_price,ask_price,count,rolling_base_volume,bid_ask_percent_change,bid_ask_volume_percent_change,close,rolling_quote_volume,symbol,quote_asset
date,,,,,,,,,,,,,,
2022-12-09 23:22:30,CVC,2.731092,0.000,0.000,0.000000,0.000000,5418,9.729966e+05,0.416667,4.578578,0.09780,9.729966e+05,CVC,CVC
2022-12-09 23:22:45,MLN,0.468823,5.758,2.226,0.001251,0.001255,2040,2.128997e+05,0.232666,89.477238,21.43000,2.128997e+05,MLN,MLN
2022-12-09 23:24:15,MULTI,-3.588517,104.446,6.329,0.000235,0.000236,6025,6.967600e+05,0.247525,94.286617,4.03000,6.967600e+05,MULTI,MULTI
2022-12-09 23:24:30,BADGER,-1.136364,1599.660,3297.450,2.610000,2.620000,2654,5.996823e+05,0.381679,49.155491,2.61000,5.996823e+05,BADGER,BADGER
2022-12-09 23:24:30,MIR,-0.990923,1296.500,1136.700,0.130890,0.131040,8084,8.227343e+05,0.114469,91.304609,0.13089,8.227343e+05,MIR,MIR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-09 23:27:30,ALPINE,3.185854,20.000,73.930,0.000158,0.000158,71988,9.855773e+06,0.051666,61.790766,2.70770,9.855773e+06,ALPINE,ALPINE
2022-12-09 23:27:30,APT,-2.456233,113.530,41.650,25.190000,25.280000,70898,2.082030e+07,0.004168,97.898230,4.79730,2.082030e+07,APT,APT
2022-12-09 23:27:30,GALA,-2.349375,194932.000,69581.000,0.037840,0.038280,49202,1.147826e+07,0.038805,74.917343,0.02577,1.147826e+07,GALA,GALA


In [4]:
output_15s = crypto_loggers['output_15s']
df_output_15s = output_15s.dataset
df_output_15s

symbol,1INCH,AAVE,ACA,ACH,ACM,ADA,ADX,AERGO,AGIX,AGLD,...,XTZ,XVG,XVS,YFI,YFII,YGG,ZEC,ZEN,ZIL,ZRX
,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-12-08 22:36:15,4.106739e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.0000,3.234452e+07,0.0000,231093.80,3119472.72,0.000000e+00,...,1.897954e+06,243779.628846,924760.5627,3.341062e+06,1.233795e+07,4.332638e+06,2.088459e+06,1.667089e+06,2.631949e+06,999124.6958
2022-12-08 22:36:30,4.106840e+06,6.279692e+06,0.000000e+00,4.842188e+06,0.0000,3.235693e+07,781471.2857,0.00,0.00,0.000000e+00,...,1.896273e+06,243779.628846,924760.5627,3.341062e+06,1.235437e+07,4.331130e+06,2.088459e+06,1.667089e+06,2.632156e+06,999124.6958
2022-12-08 22:36:45,0.000000e+00,6.302554e+06,2.959579e+06,4.842565e+06,0.0000,0.000000e+00,0.0000,0.00,3119321.72,9.214647e+06,...,1.896273e+06,243779.628846,924760.5627,3.341748e+06,1.235391e+07,4.329368e+06,2.088459e+06,1.667089e+06,2.632165e+06,999124.6958
2022-12-08 22:37:00,0.000000e+00,6.302499e+06,0.000000e+00,0.000000e+00,0.0000,3.237697e+07,0.0000,0.00,3118511.60,9.215171e+06,...,1.896273e+06,243779.628846,924760.5627,3.337181e+06,1.235391e+07,4.329522e+06,2.088380e+06,1.667089e+06,2.632177e+06,999124.6958
2022-12-08 22:37:15,4.118171e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.0000,0.000000e+00,0.0000,0.00,0.00,0.000000e+00,...,1.898237e+06,243779.628846,924760.5627,3.337181e+06,1.235391e+07,4.329676e+06,2.088321e+06,1.667089e+06,2.631122e+06,999124.6958
2022-12-08 22:37:30,4.118122e+06,6.296173e+06,0.000000e+00,0.000000e+00,0.0000,3.240398e+07,0.0000,0.00,3118580.32,0.000000e+00,...,1.898237e+06,243779.628846,924760.5627,3.337151e+06,1.235025e+07,4.329725e+06,2.088318e+06,1.667089e+06,2.631122e+06,999124.6958
2022-12-08 22:37:45,4.109026e+06,0.000000e+00,0.000000e+00,4.842540e+06,252698.5890,0.000000e+00,0.0000,0.00,3118128.36,0.000000e+00,...,1.897437e+06,243779.628846,924760.5627,3.337121e+06,1.235025e+07,4.329725e+06,2.088203e+06,1.667298e+06,2.630757e+06,999124.6958
2022-12-08 22:38:00,4.108709e+06,6.306092e+06,0.000000e+00,0.000000e+00,0.0000,3.241600e+07,0.0000,0.00,3118040.80,0.000000e+00,...,1.898058e+06,243753.063036,924760.5627,3.337121e+06,1.234233e+07,4.329253e+06,2.086573e+06,1.667298e+06,2.630970e+06,996536.7006


In [5]:
output_1min = crypto_loggers['output_1min']
df_output_1min = output_1min.dataset
df_output_1min

symbol,1INCH,AAVE,ACA,ACH,ACM,ADA,ADX,AERGO,AGIX,AGLD,...,XTZ,XVG,XVS,YFI,YFII,YGG,ZEC,ZEN,ZIL,ZRX
,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-12-01 14:30:00,1544.1816,1411.0824,1840.423332,2152.51894,88.9278,448025.06205,492.6350,0.000002,0.000000e+00,2099.4768,...,2.774007e+06,510661.195091,921426.611946,4.053938e+06,3.507638e+06,3.964233e+06,6.067457e+06,9.327093e+05,6.027409e+06,1.523769e+06
2022-12-01 14:31:00,26.1612,8318.8215,679.811419,60.30990,2234.5083,39424.97465,2122.8090,0.000000,3.300053e-06,200.0172,...,2.774441e+06,510627.963946,921426.703646,4.053390e+06,3.506929e+06,3.965054e+06,6.071264e+06,9.332758e+05,6.032084e+06,1.524053e+06
2022-12-01 14:32:00,1863.8436,10858.4320,0.000000,6134.63120,2946.1184,477629.81476,464.3860,0.000011,8.094207e-06,1063.7868,...,2.777059e+06,512495.192749,921592.102671,4.058010e+06,3.508628e+06,3.968438e+06,6.072017e+06,9.334009e+05,6.040655e+06,1.525250e+06
2022-12-01 14:33:00,1007.0126,20559.5520,17.119125,50.30576,54.6066,61359.17916,3777.2358,0.000016,4.108958e-08,659.5848,...,2.773702e+06,512781.733949,920515.427071,4.058514e+06,3.510288e+06,3.970170e+06,6.072204e+06,9.284682e+05,6.041703e+06,1.525210e+06
2022-12-01 14:34:00,510.0722,558.4640,27.747000,5165.99280,1846.3362,50465.99720,501.3657,0.000009,3.315292e-07,452.6457,...,2.768865e+06,521297.717155,919960.767506,4.058147e+06,3.508706e+06,3.970425e+06,6.067146e+06,9.296717e+05,6.038036e+06,1.515155e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-08 22:47:00,0.0000,0.0000,0.000000,0.00000,0.0000,0.00000,0.0000,0.000000,0.000000e+00,0.0000,...,1.903026e+06,243171.459543,926257.567200,3.357128e+06,1.233840e+07,4.336486e+06,2.053665e+06,1.668977e+06,2.623829e+06,9.854398e+05
2022-12-08 22:48:00,0.0000,0.0000,0.000000,0.00000,0.0000,0.00000,0.0000,0.000000,0.000000e+00,0.0000,...,1.904344e+06,243171.459543,926257.567200,3.355775e+06,1.231517e+07,4.333609e+06,2.040739e+06,1.668908e+06,2.621100e+06,9.832737e+05


In [6]:
output_30min = crypto_loggers['output_30min']
df_output_30min = output_30min.dataset
df_output_30min

symbol,1INCH,AAVE,ACA,ACH,ACM,ADA,ADX,AERGO,AGIX,AGLD,...,XTZ,XVG,XVS,YFI,YFII,YGG,ZEC,ZEN,ZIL,ZRX
,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-12-01 14:30:00,16280.6339,99287.8860,4735.164112,2.006008e+04,7565.1094,1.973748e+06,11020.4056,0.000058,0.000032,12408.4908,...,2.766412e+06,514079.120735,925471.131550,4.022432e+06,3.510459e+06,3.977199e+06,6.073680e+06,9.293989e+05,6.053766e+06,1.515691e+06
2022-12-01 15:00:00,291327.7298,432824.1538,6317.753416,4.106377e+04,10217.8481,2.064721e+06,34798.0441,0.000193,0.000106,30580.5454,...,2.860257e+06,525004.095863,911516.376559,4.006096e+06,3.578711e+06,3.835199e+06,6.120475e+06,9.847650e+05,6.183344e+06,1.499692e+06
2022-12-01 15:30:00,116616.3600,381569.5493,10907.840154,1.619974e+06,33710.9834,1.298377e+06,62440.4807,0.000085,0.000093,38221.3882,...,2.893296e+06,512570.960992,893961.946447,4.010333e+06,3.625736e+06,3.605515e+06,6.116678e+06,1.066800e+06,6.275987e+06,1.517085e+06
2022-12-01 16:00:00,149867.6923,142302.6324,3856.377960,9.080692e+05,37432.6971,1.010508e+06,21194.3114,0.000028,0.000018,13217.3437,...,2.884893e+06,500406.798596,883007.218277,3.859485e+06,3.630978e+06,3.458079e+06,6.241141e+06,1.076179e+06,6.289304e+06,1.509414e+06
2022-12-01 16:30:00,387106.4708,166894.6047,2000.517091,1.844787e+06,23197.7092,7.787929e+05,19955.1329,0.000042,0.000201,26309.2957,...,2.880471e+06,494658.970378,834430.093640,3.736899e+06,3.599136e+06,3.528254e+06,6.254834e+06,1.094720e+06,6.354688e+06,1.497935e+06
2022-12-01 17:00:00,45034.6541,161385.7819,4912.109800,5.013302e+05,36739.4590,5.594799e+05,13896.6055,0.000046,0.000174,27752.1635,...,2.905167e+06,489072.778830,825386.239028,3.750083e+06,3.563515e+06,3.811273e+06,6.239159e+06,1.106698e+06,6.328796e+06,1.470783e+06
2022-12-01 17:30:00,63046.8470,160722.2428,2344.598811,1.028638e+05,35864.1284,3.744117e+05,17770.0274,0.000007,0.000085,22935.2156,...,2.814909e+06,483500.213817,797598.135764,3.434595e+06,3.414332e+06,3.754591e+06,5.983229e+06,1.094789e+06,6.161119e+06,1.396808e+06
2022-12-01 18:00:00,26597.1481,77695.0551,2523.963809,1.769454e+05,58509.7846,9.358630e+05,21317.2627,0.000009,0.000033,5082.6038,...,2.742352e+06,483849.551342,808415.484250,3.402979e+06,3.386781e+06,3.762759e+06,5.844010e+06,1.095653e+06,6.093627e+06,1.385387e+06


In [7]:
output_1h = crypto_loggers['output_1h']
df_output_1h = output_1h.dataset
df_output_1h

symbol,1INCH,AAVE,ACA,ACH,ACM,ADA,ADX,AERGO,AGIX,AGLD,...,XTZ,XVG,XVS,YFI,YFII,YGG,ZEC,ZEN,ZIL,ZRX
,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-12-01 14:00:00,1.883395e+05,497239.8704,33350.337272,9.753389e+04,30059.1750,3.992064e+06,77816.4732,0.000461,0.000085,82273.5288,...,2.806332e+06,512094.228628,910138.984458,3.999675e+06,3.506795e+06,3.978699e+06,6.080974e+06,9.319609e+05,6.112219e+06,1.513400e+06
2022-12-01 15:00:00,3.437254e+05,795941.3660,13880.259516,2.324136e+06,63001.9572,3.427301e+06,95900.9541,0.000108,0.000182,62607.6468,...,2.911926e+06,514919.886865,894002.960227,3.910652e+06,3.648616e+06,3.570405e+06,6.165277e+06,1.072284e+06,6.279844e+06,1.509425e+06
2022-12-01 16:00:00,5.130899e+05,298929.6155,8913.264582,2.350338e+06,53929.3458,1.517700e+06,32295.5144,0.000066,0.000353,40194.6667,...,2.883595e+06,493935.857934,826124.387072,3.743536e+06,3.612649e+06,3.664785e+06,6.220869e+06,1.098003e+06,6.357607e+06,1.495646e+06
2022-12-01 17:00:00,9.076944e+04,286678.0206,4706.585616,3.207617e+05,85336.5460,8.481206e+05,30151.2292,0.000046,0.000138,50167.2504,...,2.768890e+06,487630.777598,799754.471366,3.412397e+06,3.405174e+06,3.758396e+06,5.880971e+06,1.093223e+06,6.148550e+06,1.403562e+06
2022-12-01 18:00:00,7.432357e+04,236084.5566,3486.342975,2.780377e+05,34132.2228,1.487998e+06,26712.0635,0.000022,0.000058,12907.1943,...,2.598706e+06,482349.486027,707640.405406,3.312432e+06,3.302964e+06,3.753788e+06,6.145374e+06,1.046349e+06,5.630978e+06,1.314863e+06
2022-12-01 19:00:00,1.242107e+05,230479.5948,45163.706133,1.949574e+05,56352.5292,7.899428e+05,22744.7418,0.000082,0.000043,30632.5424,...,2.407809e+06,451514.831570,691822.333269,3.165584e+06,3.165405e+06,3.191274e+06,6.045050e+06,1.029887e+06,5.478740e+06,1.309267e+06
2022-12-01 20:00:00,1.115960e+05,102650.5272,8820.260985,2.661634e+05,10250.6975,1.094265e+06,39073.6672,0.000067,0.000175,13287.9201,...,2.345358e+06,443909.598473,679526.602219,3.149410e+06,3.038972e+06,2.740270e+06,5.982433e+06,1.059589e+06,5.386862e+06,1.239805e+06
2022-12-01 21:00:00,2.428385e+04,66235.1846,6280.606824,9.211913e+03,0.0000,4.655759e+05,1190.2678,0.000000,0.000004,0.0000,...,2.202862e+06,431861.390494,661056.498104,3.063448e+06,2.960447e+06,2.635233e+06,5.903990e+06,1.044518e+06,5.234980e+06,1.155312e+06


In [8]:
output_1d = crypto_loggers['output_1d']
df_output_1d = output_1d.dataset
df_output_1d

symbol,1INCH,AAVE,ACA,ACH,ACM,ADA,ADX,AERGO,AGIX,AGLD,...,XTZ,XVG,XVS,YFI,YFII,YGG,ZEC,ZEN,ZIL,ZRX
,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-10-06,1.962548e+06,7.816504e+06,9.915393e+05,4.159341e+06,1.510782e+06,6.451355e+07,6.889733e+05,0.014319,0.001216,2.436420e+06,...,4.607039e+06,2.374232e+05,1.276883e+06,2.565424e+06,7.131460e+05,2.345798e+06,5.798678e+06,9.001644e+05,6.618073e+06,6.370141e+05
2022-10-07,1.867650e+07,5.728082e+06,5.304131e+05,6.271438e+06,1.023109e+06,5.772288e+07,8.278553e+05,0.031345,0.001531,6.032983e+05,...,2.516770e+06,2.214177e+05,3.621004e+06,1.882804e+06,7.689245e+05,1.601181e+06,3.268516e+06,5.785303e+05,6.680246e+06,7.114988e+05
2022-10-08,4.667957e+06,2.210234e+06,2.598952e+05,2.492753e+06,3.272902e+05,3.627886e+07,5.695966e+05,0.006721,0.001852,7.113260e+05,...,1.233464e+06,2.125255e+05,6.482011e+05,1.022934e+06,5.864172e+05,6.431035e+06,2.343833e+06,3.853899e+05,4.354008e+06,1.420885e+06
2022-10-09,2.106350e+06,2.335111e+06,3.234287e+05,2.446181e+06,2.949945e+05,3.305531e+07,4.596247e+05,0.018170,0.000460,1.842133e+06,...,1.154256e+06,1.412745e+05,1.149393e+06,6.362114e+05,2.731952e+06,8.434631e+06,2.903015e+06,3.704477e+05,1.024151e+07,1.032559e+06
2022-10-10,2.996440e+06,3.063826e+06,4.501791e+05,3.637729e+06,1.236916e+06,7.161114e+07,1.349220e+06,0.004230,0.001168,1.283936e+06,...,2.354251e+06,1.966290e+05,4.747862e+05,1.413566e+06,1.715331e+06,4.952464e+06,3.371167e+06,1.214529e+06,6.819190e+06,8.920935e+05
2022-10-11,2.125461e+06,4.712442e+06,4.513696e+05,8.291305e+05,1.228223e+06,9.607438e+07,1.935635e+06,0.003603,0.000963,1.128402e+06,...,2.812294e+06,1.681242e+06,6.452080e+05,2.229223e+06,3.498008e+06,2.211741e+06,2.699886e+06,9.450011e+05,7.500061e+06,9.035241e+05
2022-10-12,1.123827e+06,3.075026e+06,6.566533e+05,1.680225e+06,5.092616e+05,6.300752e+07,1.373768e+06,0.001839,0.023891,4.248593e+05,...,1.847150e+06,7.565095e+05,6.366835e+05,1.059110e+06,1.060731e+06,1.089623e+06,2.317108e+06,4.524865e+05,3.039741e+06,3.741372e+05
2022-10-13,6.590568e+06,1.524694e+07,2.372160e+06,2.391955e+06,1.508736e+06,1.714384e+08,4.033211e+06,0.008126,0.094941,9.573303e+05,...,1.069227e+07,6.799174e+05,8.089423e+05,4.469345e+06,2.148870e+06,4.853106e+06,6.560353e+06,1.502653e+06,1.144101e+07,1.859398e+06


In [9]:
df = df_output_1d.copy()
df

symbol,1INCH,AAVE,ACA,ACH,ACM,ADA,ADX,AERGO,AGIX,AGLD,...,XTZ,XVG,XVS,YFI,YFII,YGG,ZEC,ZEN,ZIL,ZRX
,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-10-06,1.962548e+06,7.816504e+06,9.915393e+05,4.159341e+06,1.510782e+06,6.451355e+07,6.889733e+05,0.014319,0.001216,2.436420e+06,...,4.607039e+06,2.374232e+05,1.276883e+06,2.565424e+06,7.131460e+05,2.345798e+06,5.798678e+06,9.001644e+05,6.618073e+06,6.370141e+05
2022-10-07,1.867650e+07,5.728082e+06,5.304131e+05,6.271438e+06,1.023109e+06,5.772288e+07,8.278553e+05,0.031345,0.001531,6.032983e+05,...,2.516770e+06,2.214177e+05,3.621004e+06,1.882804e+06,7.689245e+05,1.601181e+06,3.268516e+06,5.785303e+05,6.680246e+06,7.114988e+05
2022-10-08,4.667957e+06,2.210234e+06,2.598952e+05,2.492753e+06,3.272902e+05,3.627886e+07,5.695966e+05,0.006721,0.001852,7.113260e+05,...,1.233464e+06,2.125255e+05,6.482011e+05,1.022934e+06,5.864172e+05,6.431035e+06,2.343833e+06,3.853899e+05,4.354008e+06,1.420885e+06
2022-10-09,2.106350e+06,2.335111e+06,3.234287e+05,2.446181e+06,2.949945e+05,3.305531e+07,4.596247e+05,0.018170,0.000460,1.842133e+06,...,1.154256e+06,1.412745e+05,1.149393e+06,6.362114e+05,2.731952e+06,8.434631e+06,2.903015e+06,3.704477e+05,1.024151e+07,1.032559e+06
2022-10-10,2.996440e+06,3.063826e+06,4.501791e+05,3.637729e+06,1.236916e+06,7.161114e+07,1.349220e+06,0.004230,0.001168,1.283936e+06,...,2.354251e+06,1.966290e+05,4.747862e+05,1.413566e+06,1.715331e+06,4.952464e+06,3.371167e+06,1.214529e+06,6.819190e+06,8.920935e+05
2022-10-11,2.125461e+06,4.712442e+06,4.513696e+05,8.291305e+05,1.228223e+06,9.607438e+07,1.935635e+06,0.003603,0.000963,1.128402e+06,...,2.812294e+06,1.681242e+06,6.452080e+05,2.229223e+06,3.498008e+06,2.211741e+06,2.699886e+06,9.450011e+05,7.500061e+06,9.035241e+05
2022-10-12,1.123827e+06,3.075026e+06,6.566533e+05,1.680225e+06,5.092616e+05,6.300752e+07,1.373768e+06,0.001839,0.023891,4.248593e+05,...,1.847150e+06,7.565095e+05,6.366835e+05,1.059110e+06,1.060731e+06,1.089623e+06,2.317108e+06,4.524865e+05,3.039741e+06,3.741372e+05
2022-10-13,6.590568e+06,1.524694e+07,2.372160e+06,2.391955e+06,1.508736e+06,1.714384e+08,4.033211e+06,0.008126,0.094941,9.573303e+05,...,1.069227e+07,6.799174e+05,8.089423e+05,4.469345e+06,2.148870e+06,4.853106e+06,6.560353e+06,1.502653e+06,1.144101e+07,1.859398e+06


In [10]:
df['BTC']

,base_volume,close,high,low,open,quote_volume,rolling_base_volume,rolling_quote_volume
date,,,,,,,,
2022-10-06,9.453219e+09,19960.670000,20456.600000,19853.000000,20158.260000,9.540715e+09,9.453219e+09,9.540715e+09
2022-10-07,6.657955e+09,19530.090000,20068.820000,19320.000000,19960.670000,6.718068e+09,6.657955e+09,6.718068e+09
2022-10-08,3.093736e+09,19417.960000,19627.380000,19237.140000,19530.090000,3.101588e+09,3.093736e+09,3.101588e+09
2022-10-09,3.467721e+09,19439.020000,19558.000000,19316.040000,19416.520000,3.469331e+09,3.467721e+09,3.469331e+09
2022-10-10,6.314628e+09,19131.870000,19525.000000,19020.250000,19439.960000,6.363682e+09,6.314628e+09,6.363682e+09
2022-10-11,7.113666e+09,19060.000000,19268.090000,18860.000000,19131.870000,7.115585e+09,7.113666e+09,7.115585e+09
2022-10-12,6.211497e+09,19155.530000,19238.310000,18965.880000,19060.000000,6.197095e+09,6.211497e+09,6.197095e+09
2022-10-13,1.137237e+10,19375.130000,19513.790000,18190.000000,19155.100000,1.110680e+10,1.137237e+10,1.110680e+10
2022-10-14,1.056832e+10,19176.930000,19951.870000,19070.370000,19375.580000,1.077548e+10,1.056832e+10,1.077548e+10


In [ ]:
import pandas_ta as ta

def filter_in_market(function, dataset):
    def f(x):
        x = x.loc[:,~x.columns.duplicated()]
        return function(x)
    tickers_list = dataset.columns.get_level_values(0).unique().tolist()
    return pd.Series([ticker for ticker in tickers_list if f(dataset[ticker])], dtype='str')

def get_relative_volume_levels_smoothed_trigger(data, average1=26, average2=14):
    volume = data['volume']
    volume_average = ta.sma(close=volume, length=average1)
    relative_volume = volume / volume_average
    smoothed_relative_volume = ta.sma(close=relative_volume, length=average2)
    return (smoothed_relative_volume > threshold).iat[-1]

def get_relative_volume_levels_at_time_smoothed_thresholded(data):
    try:
        volume = data['volume']
        #volume = volume.groupby(pd.Grouper(freq='D')).cumsum()
        cum_volume = volume.groupby(pd.Grouper(freq='24h')).cumsum()
        #volume = volume.groupby(pd.Grouper(freq='60m')).cumsum()
        cum_rvol = (cum_volume / cum_volume.shift(1)).fillna(method='pad')
        rvol = (volume / volume.shift(1)).fillna(method='pad')
        bar_up = (data['close'] > data['open'])
        bar_up |= (data['close'] == data['open']) & (data['close'].diff() > 0)
        bar_up = bar_up.astype(int)
        bar_up = bar_up * 2 - 1
        #rvol *= bar_up
        cum_rvol_dir = cum_rvol * bar_up
        rvol_dir = rvol * bar_up
        rvol_indicator = ta.hma(rvol, length=14, talib=True)
        rvol_dir_indicator = ta.hma(rvol_dir, length=14, talib=True)
        cum_rvol_indicator = ta.hma(cum_rvol, length=14, talib=True)
        cum_rvol_dir_indicator = ta.hma(cum_rvol_dir, length=14, talib=True)
        rvol_indicator = rvol_indicator.rename('relative_volume_levels_smoothed')
        rvol_dir_indicator = rvol_dir_indicator.rename('relative_volume_levels_dir_smoothed')
        cum_rvol_indicator = cum_rvol_indicator.rename('cum_relative_volume_levels_smoothed')
        cum_rvol_dir_indicator = cum_rvol_dir_indicator.rename('cum_relative_volume_levels_dir_smoothed')
        #threshold = (ta.sma(rvol, length=100, talib=True) + ta.stdev(rvol, length=100, talib=True))
        threshold_dir = 0
        threshold = 2
        rvol_thresholded = (rvol_indicator > threshold).iat[-1]
        rvol_dir_thresholded = (rvol_dir_indicator > threshold_dir).iat[-1]
        cum_rvol_thresholded = (cum_rvol_indicator > threshold).iat[-1]
        cum_rvol_dir_thresholded = (cum_rvol_dir_indicator > threshold_dir).iat[-1]
        trigger = (rvol_thresholded | rvol_dir_thresholded | cum_rvol_thresholded | cum_rvol_dir_thresholded)
    except Exception as e:
        print('rvol exception:', e)
        trigger = False
    return trigger

def get_positive_trend_strength_trigger(data):
    ADX = data.ta.adx(talib=True)
    return (ADX['ADX_14'] < 0.20).iloc[-3] & (ADX['ADX_14'] > 0.20).iat[-2]

def get_not_negative_trend_strength_trigger(data):
    ADX = data.ta.adx(length=14, lensig=8, talib=True)
    return ((ADX['DMP_14'] > ADX['DMN_14']) & (ADX['ADX_14'] > 0.30)).iat[-1]

def get_not_negative_rebound_trigger(data):
    CCI = data.ta.cci(length=22, talib=True)
    MFI = data.ta.mfi(length=11, talib=True)
    return ((CCI > 0) | (MFI > 20)).iat[-1]

def get_positive_choppiness_trigger(data):
    CHOP = data.ta.chop(talib=True)
    return CHOP.iat[-1] < 38.2

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram > histogram.shift(1)) | \
            (MACD['MACD_12_26_9'] > MACD['MACDs_12_26_9'])).iat[-1]

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram.iloc[-2] > histogram.iat[-2]) or \
            (MACD['MACD_12_26_9'].iat[-1] > MACD['MACDs_12_26_9'].iat[-1]))

def get_not_square_wave_triggers(data, multiplier_schedule):
    triggers = True
    for multiplier in multiplier_schedule:
        period_1 = -4 * multiplier
        uniques_1 = 2 * multiplier
        square_wave_trigger_1 = (data.iloc[period_1:]['close'].unique().size < uniques_1)
        if square_wave_trigger_1:
            triggers = False
            break
        else:
            period_2 = -15 * multiplier
            uniques_2 = 6 * multiplier
            square_wave_trigger_2 = (data.iloc[period_2:]['close'].unique().size < uniques_2)
            if square_wave_trigger_2:
                triggers = False
                break
    return triggers

def get_minute_not_square_wave_triggers(data):
    return get_not_square_wave_triggers(data, multiplier_schedule=[1, 2, 3, 5, 10, 15, 20, 45])

def get_hourly_not_square_wave_triggers(data):
    return get_not_square_wave_triggers(data, multiplier_schedule=[1])

def get_daily_not_square_wave_triggers(data):
    return get_not_square_wave_triggers(data, multiplier_schedule=[1])

def get_daily_volume_minimum_trigger(data):
    return (data['volume'] > 1000000).iat[-1]

def get_daily_volume_change_trigger(data):
    return ((data['volume'].pct_change(1) * 100) > 300).iat[-1]

def get_minute_daily_volume_minimum_trigger(data):
    return (data['rolling_base_volume'] > 1000000).iat[-1]

def get_minute_daily_volume_change_trigger(data):
    return ((data['rolling_base_volume'].pct_change(1440) * 100) > 300).iat[-1]

def get_rising_volume_trigger(data):
    return (data['rolling_base_volume'].diff(1) > 0).iat[-1]

In [ ]:
df = df_1d.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered_1 = set(filter_in_market(get_daily_not_square_wave_triggers, df).tolist())
filtered_2 = set(filter_in_market(get_daily_volume_minimum_trigger, df).tolist())
filtered_3 = set(filter_in_market(get_relative_volume_levels_smoothed_trigger, df).tolist())
filtered_1d = pd.Series(list(filtered_1 & filtered_2 & filtered_3))
filtered_1d

In [ ]:
df = df_1h.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered_1 = set(filter_in_market(get_hourly_not_square_wave_triggers, df).tolist())
filtered_2 = set(filter_in_market(get_relative_volume_levels_at_time_smoothed_thresholded, df).tolist())
filtered_1h = pd.Series(list(filtered_1 & filtered_2))
filtered_1h

In [ ]:
from cryptocurrency.indicators import filter_in_market, screen_one
df = df_1min.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered = filter_in_market(screen_one, df).tolist()
filtered

In [ ]:
df = df_1min.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered_1 = set(filter_in_market(get_minute_not_square_wave_triggers, df).tolist())
filtered_2 = set(filter_in_market(get_minute_daily_volume_minimum_trigger, df).tolist())
filtered_3 = set(filter_in_market(get_minute_daily_volume_change_trigger, df).tolist())
filtered_4 = set(filter_in_market(get_rising_volume_trigger, df).tolist())
filtered_1min = pd.Series(list(filtered_1 & filtered_2 & filtered_3 & filtered_4))
filtered_1min

In [ ]:
list(set(filtered_1h.tolist()) & set(filtered_1d.tolist()))

In [ ]:
list(set(filtered_1min.tolist()) & set(filtered_1h.tolist()))

In [ ]:
list(set(filtered_1min.tolist()) & set(filtered_1d.tolist()))

In [ ]:
list(set(filtered_1min.tolist()) & set(filtered_1h.tolist()) & set(filtered_1d.tolist()))

In [ ]:
import pandas as pd
crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df_15s = pd.read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df_15s.index = pd.DatetimeIndex(df_15s.index)
df_15s

In [ ]:
import pandas as pd

crypto_exchange_info = 'crypto_logs/crypto_exchange_info.txt'
exchange_info = pd.read_csv(crypto_exchange_info, header=0, index_col=0)
exchange_info

In [ ]:
crypto_loggers['input_15s'].dataset

In [ ]:
dataset_screened = crypto_loggers['input_15s'].screen(crypto_loggers['input_15s'].dataset)
dataset_screened

In [11]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import get_timezone_offset_in_seconds
from cryptocurrency.conversion_table import get_conversion_table

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info
offset_s = get_timezone_offset_in_seconds()

#conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, 
#                                        offset_s=offset_s, as_pair=False)
#conversion_table.sort_values(by='rolling_base_volume', ascending=False).reset_index(drop=True).head(50)

In [17]:
from cryptocurrency.conversion import convert_price, get_base_asset_from_pair
from cryptocurrency.conversion import get_quote_asset_from_pair, get_shortest_pair_path_between_assets
from cryptocurrency.conversion import convert_price_from_shortest_path
import datetime
import pandas as pd

def get_conversion_table_from_binance(client, exchange_info, offset_s=0, dump_raw=False):
    conversion_table = pd.DataFrame(client.get_ticker())
    conversion_table = conversion_table[conversion_table['symbol'].isin(exchange_info['symbol'])]

    conversion_table['base_asset'] = \
        conversion_table['symbol'].apply(lambda x: get_base_asset_from_pair(x, exchange_info=exchange_info))
    conversion_table['quote_asset'] = \
        conversion_table['symbol'].apply(lambda x: get_quote_asset_from_pair(x, exchange_info=exchange_info))

    conversion_table = \
        conversion_table.rename(
            columns={'openPrice': 'open', 'highPrice': 'high', 'lowPrice': 'low', 'lastPrice': 'close', 
                     'lastQty': 'last_volume', 'volume': 'rolling_base_volume', 'quoteVolume': 'rolling_quote_volume', 
                     'bidPrice': 'bid_price', 'askPrice': 'ask_price', 'bidQty': 'bid_volume', 'askQty': 'ask_volume', 
                     'firstId': 'first_ID', 'lastId': 'last_ID', 'openTime': 'open_time', 'closeTime': 'close_time', 
                     'prevClosePrice': 'close_shifted', 'weightedAvgPrice': 'weighted_average_price', 
                     'priceChange': 'price_change', 'priceChangePercent': 'price_change_percent'})

    if dump_raw:
        conversion_table.to_csv('crypto_logs/conversion_table.txt')

    conversion_table[['open', 'high', 'low', 'close', 'close_shifted', 'rolling_base_volume', 
                      'rolling_quote_volume', 'bid_price', 'ask_price', 'bid_volume', 'ask_volume', 
                      'price_change_percent']] = \
        conversion_table[['open', 'high', 'low', 'close', 'close_shifted', 'rolling_base_volume', 
                          'rolling_quote_volume', 'bid_price', 'ask_price', 'bid_volume', 'ask_volume', 
                          'price_change_percent']].astype(float)

    conversion_table[['close_time', 'count']] = \
        conversion_table[['close_time', 'count']].astype(int)
    conversion_table['close_time'] = (conversion_table['close_time'] + offset_s * 1000)
    conversion_table['close_time'] /= 1000
    conversion_table['close_time'] = conversion_table['close_time'].apply(datetime.datetime.fromtimestamp)
    conversion_table['close_time'] = pd.DatetimeIndex(conversion_table['close_time'])
    return conversion_table.sort_values(by='close_time')

def process_conversion_table(conversion_table, exchange_info, as_pair=False, minimal=True, 
                             extra_minimal=True, super_extra_minimal=True, convert_to_USDT=True):
    """
    Fetches and prepares data used to calculate prices, volumes and other stats.
    :param client: object from python-binance useful for calling client.get_ticker().
    :param exchange_info: Pre-calculated exchange information on all tickers.
    :return: pd.DataFrame containing all preprocessed conversion table info.
    :column is_shorted: is the symbol made from inversion.
    :column symbol: concatenated string made from base_asset and quote_asset.
    :column shorted_symbol: symbol with inverted base_asset and quote_asset.
    :column base_asset: asset on the left.
    :column quote_asset: asset on the right.
    :column price_change: (close - open).
    :column price_change_percent: (((close - open) / open) * 100).
    :column USDT_price_change: (USDT_close - USDT_open).
    :column USDT_price_change_percent: (((USDT_close - USDT_open) / USDT_open) * 100).
    :column weighted_average_price: weighted average price.
    :column close_shifted: close price of the previous day.
    :column open: open price of the day.
    :column high: high price of the day.
    :column low: low price of the day.
    :column close: close price of the day.
    :column last_volume: volume of the last price update.
    :column USDT_bid_price: USDT-converted bid price.
    :column USDT_ask_price: USDT-converted ask price.
    :column USDT_bid_volume: USDT-converted bid volume.
    :column USDT_ask_volume: USDT-converted ask volume.
    :column bid_price: price of the bid.
    :column bid_volume: volume of the bid at bid_price.
    :column ask_price: price of the ask.
    :column ask_volume: volume of the ask at ask_price.
    :column rolling_base_volume: rolling_base_volume given by the API.
    :column rolling_quote_volume: rolling_quote_volume given by the API.
    :column open_time: close_time minus 24 hours.
    :column close_time: time from epoch in milliseconds of the last price update.
    :column first_ID: transaction ID from 1 day ago.
    :column last_ID: latest transaction ID.
    :column count: value calculated by subtracting first_ID from last_ID.
    :column USDT_open: USDT-converted open price.
    :column USDT_high: USDT-converted high price.
    :column USDT_low: USDT-converted low price.
    :column USDT_price: USDT-converted close price.
    :column rolling_USDT_base_volume: USDT-converted rolling_base_volume.
    :column rolling_USDT_quote_volume: USDT-converted rolling_quote_volume.
    :column rolling_traded_volume: sum by base_asset of all USDT-converted volumes.
    :column importance: rolling_USDT_base_volume divided by rolling_traded_volume.
    :column traded_price: sum by base_asset of all (close prices times importance).
    :column traded_bid_price: sum by base_asset of all (bid prices times importance).
    :column traded_ask_price: sum by base_asset of all (ask prices times importance).
    :column bid_ask_percent_change: ((ask_price - bid_price) / ask_price) * 100).
    :column bid_ask_volume_percent_change: ((bid_volume / (bid_volume + ask_volume)) * 100).
    :column traded_bid_ask_percent_change: ((traded_ask_price - traded_bid_price) / traded_ask_price) * 100).
    :column traded_bid_ask_volume_percent_change: ((traded_bid_volume / (traded_bid_volume + traded_ask_volume)) * 100).
    """
    if as_pair:
        super_extra_minimal = False
    else:
        convert_to_USDT = True

    if super_extra_minimal:
        extra_minimal = True

    if extra_minimal:
        minimal = True

    if minimal:
        conversion_table = \
            conversion_table[['symbol', 'base_asset', 'quote_asset', 'open', 'close', 'rolling_base_volume', 
                              'rolling_quote_volume', 'count', 'bid_price', 'ask_price', 'bid_volume', 
                              'ask_volume', 'close_time', 'price_change_percent']]

    conversion_table['rolling_base_quote_volume'] = \
        conversion_table['rolling_quote_volume'] / conversion_table['close']

    if convert_to_USDT:
        conversion_table['shortest_path'] = \
            conversion_table.apply(lambda x: get_shortest_pair_path_between_assets(
                from_asset=x['base_asset'], to_asset='USDT', exchange_info=exchange_info, 
                priority='accuracy'), axis='columns')
        if not minimal:
            conversion_table['USDT_high'] = \
                conversion_table.apply(
                    lambda x: convert_price_from_shortest_path(
                        size=(((x['high'] - x['close']) / x['close']) + 1), 
                        from_asset=x['base_asset'], to_asset='USDT', 
                        conversion_table=conversion_table, 
                        exchange_info=exchange_info, 
                        shortest_path=x['shortest_path'], 
                        key='close', priority='accuracy'), axis='columns')
            conversion_table['USDT_low'] = \
                conversion_table.apply(
                    lambda x: convert_price_from_shortest_path(
                        size=(((x['low'] - x['close']) / x['close']) + 1), 
                        from_asset=x['base_asset'], to_asset='USDT', 
                        conversion_table=conversion_table, 
                        exchange_info=exchange_info, 
                        shortest_path=x['shortest_path'], 
                        key='close', priority='accuracy'), axis='columns')
        if not super_extra_minimal:
            conversion_table['USDT_open'] = \
                conversion_table.apply(
                    lambda x: convert_price_from_shortest_path(
                        size=1, from_asset=x['base_asset'], 
                        to_asset='USDT', 
                        conversion_table=conversion_table, 
                        exchange_info=exchange_info, 
                        shortest_path=x['shortest_path'], 
                        key='open', priority='accuracy'), axis='columns')
        conversion_table['USDT_price'] = \
            conversion_table.apply(
                lambda x: convert_price_from_shortest_path(
                    size=1, from_asset=x['base_asset'], 
                    to_asset='USDT', 
                    conversion_table=conversion_table, 
                    exchange_info=exchange_info, 
                    shortest_path=x['shortest_path'], 
                    key='close', priority='accuracy'), axis='columns')
        conversion_table['rolling_USDT_base_volume'] = \
            conversion_table['rolling_base_volume'] * conversion_table['USDT_price'].astype(float)
        conversion_table['rolling_USDT_quote_volume'] = \
            conversion_table['rolling_base_quote_volume'] * conversion_table['USDT_price'].astype(float)
        conversion_table = conversion_table.drop(columns=['rolling_base_quote_volume'])

        if not extra_minimal:
            conversion_table['USDT_bid_price'] = \
                conversion_table.apply(
                    lambda x: convert_price_from_shortest_path(
                        shortest_path=x['shortest_path'], 
                        size=(((x['bid_price'] - x['close']) / x['close']) + 1), 
                        from_asset=x['base_asset'], to_asset='USDT', 
                        conversion_table=conversion_table, 
                        exchange_info=exchange_info, key='close', 
                        priority='accuracy'), axis='columns')
            conversion_table['USDT_ask_price'] = \
                conversion_table.apply(
                    lambda x: convert_price_from_shortest_path(
                        shortest_path=x['shortest_path'], 
                        size=(((x['ask_price'] - x['close']) / x['close']) + 1), 
                        from_asset=x['base_asset'], to_asset='USDT', 
                        conversion_table=conversion_table, 
                        exchange_info=exchange_info, key='close', 
                        priority='accuracy'), axis='columns')
            conversion_table['USDT_bid_volume'] = \
                conversion_table['bid_volume'] * conversion_table['USDT_bid_price'].astype(float)
            conversion_table['USDT_ask_volume'] = \
                conversion_table['ask_volume'] * conversion_table['USDT_ask_price'].astype(float)

        if super_extra_minimal:
            price_change_percent = conversion_table[['base_asset', 'price_change_percent']]
            price_change_percent = \
                price_change_percent.groupby(by='base_asset').agg(lambda x: x.iloc[x.abs().argmax()])
            price_change_percent = price_change_percent['price_change_percent']
            conversion_table['price_change_percent'] = \
                conversion_table.apply(lambda x: price_change_percent.loc[x['base_asset']], axis='columns')
        else:
            conversion_table['USDT_price_change'] = \
                (conversion_table['USDT_price'].astype(float) - conversion_table['USDT_open'].astype(float))
            conversion_table['USDT_price_change_percent'] = \
                ((conversion_table['USDT_price_change'] / conversion_table['USDT_open'].astype(float)) * 100)

        conversion_table['is_shorted'] = False

        conversion_table_swapped = conversion_table.copy()
        if minimal:
            if extra_minimal:
                if super_extra_minimal:
                    conversion_table_swapped.loc[:, ['symbol', 'price_change_percent', 'close', 'ask_price', 
                                                     'ask_volume', 'bid_price', 'bid_volume', 'open', 
                                                     'rolling_quote_volume', 'rolling_base_volume', 'close_time', 
                                                     'count', 'quote_asset', 'base_asset', 'USDT_price', 
                                                     'rolling_USDT_quote_volume', 'rolling_USDT_base_volume', 
                                                     'is_shorted']] = \
                        conversion_table_swapped.loc[:, ['symbol', 'price_change_percent', 'close', 'bid_price', 
                                                         'bid_volume', 'ask_price', 'ask_volume', 'open', 
                                                         'rolling_base_volume', 'rolling_quote_volume', 
                                                         'close_time', 'count', 'base_asset', 'quote_asset', 
                                                         'USDT_price', 'rolling_USDT_base_volume', 
                                                         'rolling_USDT_quote_volume', 'is_shorted']].values
                else:
                    conversion_table_swapped.loc[:, ['symbol', 'price_change_percent', 'close', 'ask_price', 
                                                     'ask_volume', 'bid_price', 'bid_volume', 'open', 
                                                     'rolling_quote_volume', 'rolling_base_volume', 'close_time', 
                                                     'count', 'quote_asset', 'base_asset', 'USDT_open', 
                                                     'USDT_price', 'rolling_USDT_quote_volume', 
                                                     'rolling_USDT_base_volume', 'USDT_price_change_percent', 
                                                     'is_shorted']] = \
                        conversion_table_swapped.loc[:, ['symbol', 'price_change_percent', 'close', 'bid_price', 
                                                         'bid_volume', 'ask_price', 'ask_volume', 'open', 
                                                         'rolling_base_volume', 'rolling_quote_volume', 
                                                         'close_time', 'count', 'base_asset', 'quote_asset', 
                                                         'USDT_open', 'USDT_price', 'rolling_USDT_base_volume', 
                                                         'rolling_USDT_quote_volume', 'USDT_price_change_percent', 
                                                         'is_shorted']].values
            else:
                conversion_table_swapped.loc[:, ['symbol', 'price_change_percent', 'close', 'ask_price', 
                                                 'ask_volume', 'bid_price', 'bid_volume', 'open', 
                                                 'rolling_quote_volume', 'rolling_base_volume', 'close_time', 
                                                 'count', 'quote_asset', 'base_asset', 'USDT_open', 'USDT_price', 
                                                 'rolling_USDT_quote_volume', 'rolling_USDT_base_volume', 
                                                 'USDT_ask_price', 'USDT_bid_price', 'USDT_ask_volume', 
                                                 'USDT_bid_volume', 'USDT_price_change_percent', 'is_shorted']] = \
                    conversion_table_swapped.loc[:, ['symbol', 'price_change_percent', 'close', 'bid_price', 
                                                     'bid_volume', 'ask_price', 'ask_volume', 'open', 
                                                     'rolling_base_volume', 'rolling_quote_volume', 'close_time', 
                                                     'count', 'base_asset', 'quote_asset', 'USDT_open', 'USDT_price', 
                                                     'rolling_USDT_base_volume', 'rolling_USDT_quote_volume', 
                                                     'USDT_bid_price', 'USDT_ask_price', 'USDT_bid_volume', 
                                                     'USDT_ask_volume', 'USDT_price_change_percent', 
                                                     'is_shorted']].values
        else:
            conversion_table_swapped.loc[:, ['symbol', 'price_change', 'price_change_percent', 
                                             'weighted_average_price', 'close_shifted', 'close', 
                                             'last_volume', 'ask_price', 'ask_volume', 'bid_price', 
                                             'bid_volume', 'open', 'high', 'low', 'rolling_quote_volume', 
                                             'rolling_base_volume', 'open_time', 'close_time', 'first_ID', 
                                             'last_ID', 'count', 'quote_asset', 'base_asset', 'USDT_open', 
                                             'USDT_high', 'USDT_low', 'USDT_price', 'rolling_USDT_quote_volume', 
                                             'rolling_USDT_base_volume', 'USDT_ask_price', 'USDT_bid_price', 
                                             'USDT_ask_volume', 'USDT_bid_volume', 'USDT_price_change', 
                                             'USDT_price_change_percent', 'is_shorted']] = \
                conversion_table_swapped.loc[:, ['symbol', 'price_change', 'price_change_percent', 
                                                 'weighted_average_price', 'close_shifted', 'close', 
                                                 'last_volume', 'bid_price', 'bid_volume', 'ask_price', 
                                                 'ask_volume', 'open', 'high', 'low', 'rolling_base_volume', 
                                                 'rolling_quote_volume', 'open_time', 'close_time', 'first_ID', 
                                                 'last_ID', 'count', 'base_asset', 'quote_asset', 
                                                 'USDT_open', 'USDT_high', 'USDT_low', 'USDT_price', 
                                                 'rolling_USDT_base_volume', 'rolling_USDT_quote_volume', 
                                                 'USDT_bid_price', 'USDT_ask_price', 'USDT_bid_volume', 
                                                 'USDT_ask_volume', 'USDT_price_change', 
                                                 'USDT_price_change_percent', 'is_shorted']].values

        conversion_table_swapped['symbol'] = \
            conversion_table_swapped['base_asset'] + conversion_table_swapped['quote_asset']

        if minimal:
            if extra_minimal:
                if super_extra_minimal:
                    conversion_table_swapped.loc[:, ['open', 'close', 'bid_price', 'ask_price', 'USDT_price']] = \
                        1 / conversion_table_swapped.loc[:, ['open', 'close', 'bid_price', 'ask_price', 
                                                             'USDT_price']].astype(float)
                else:
                    conversion_table_swapped.loc[:, ['open', 'close', 'bid_price', 'ask_price', 'USDT_open', 
                                                     'USDT_price', 'USDT_price_change_percent']] = \
                        1 / conversion_table_swapped.loc[:, ['open', 'close', 'bid_price', 'ask_price', 
                                                             'USDT_open', 'USDT_price', 
                                                             'USDT_price_change_percent']].astype(float)
            else:
                conversion_table_swapped.loc[:, ['open', 'close', 'bid_price', 'ask_price', 'USDT_open', 
                                                 'USDT_price', 'USDT_bid_price', 'USDT_ask_price', 
                                                 'USDT_price_change_percent']] = \
                    1 / conversion_table_swapped.loc[:, ['open', 'close', 'bid_price', 'ask_price', 'USDT_open', 
                                                         'USDT_price', 'USDT_bid_price', 'USDT_ask_price', 
                                                         'USDT_price_change_percent']].astype(float)
        else:
            conversion_table_swapped.loc[:, ['open', 'high', 'low', 'close', 'close_shifted', 'bid_price', 
                                             'ask_price', 'USDT_open', 'USDT_high', 'USDT_low', 'USDT_price', 
                                             'USDT_bid_price', 'USDT_ask_price', 'USDT_price_change', 
                                             'USDT_price_change_percent']] = \
                1 / conversion_table_swapped.loc[:, ['open', 'high', 'low', 'close', 'close_shifted', 
                                                     'bid_price', 'ask_price', 'USDT_open', 'USDT_high', 
                                                     'USDT_low', 'USDT_price', 'USDT_bid_price', 'USDT_ask_price', 
                                                     'USDT_price_change', 'USDT_price_change_percent']].astype(float)
        conversion_table_swapped['is_shorted'] = True

        conversion_table = pd.concat([conversion_table, conversion_table_swapped], join='outer', axis='index')

        traded_volume = conversion_table.groupby(by='base_asset').agg('sum')
        traded_volume = traded_volume['rolling_USDT_base_volume']
        conversion_table['rolling_traded_volume'] = \
            conversion_table.apply(lambda x: traded_volume.loc[x['base_asset']], axis='columns')
        if not extra_minimal:
            traded_bid_volume = conversion_table.groupby(by='base_asset').agg('sum')
            traded_bid_volume = traded_bid_volume['USDT_bid_volume']
            conversion_table['traded_bid_volume'] = \
                conversion_table.apply(lambda x: traded_bid_volume.loc[x['base_asset']], axis='columns')
            traded_ask_volume = conversion_table.groupby(by='base_asset').agg('sum')
            traded_ask_volume = traded_ask_volume['USDT_ask_volume']
            conversion_table['traded_ask_volume'] = \
                conversion_table.apply(lambda x: traded_ask_volume.loc[x['base_asset']], axis='columns')

        conversion_table['importance'] = \
            conversion_table['rolling_USDT_base_volume'] / conversion_table['rolling_traded_volume']
        conversion_table['importance_weighted_price'] = \
            conversion_table['USDT_price'].astype(float) * conversion_table['importance']

        if not extra_minimal:
            conversion_table['importance_weighted_bid_price'] = \
                conversion_table['USDT_bid_price'].astype(float) * conversion_table['importance']
            conversion_table['importance_weighted_ask_price'] = \
                conversion_table['USDT_ask_price'].astype(float) * conversion_table['importance']

        importance_weighted_price = conversion_table.groupby(by='base_asset').agg('sum')
        importance_weighted_price = importance_weighted_price['importance_weighted_price']
        conversion_table['traded_price'] = \
            conversion_table.apply(lambda x: importance_weighted_price.loc[x['base_asset']], axis='columns')

        if not extra_minimal:
            importance_weighted_bid_price = conversion_table.groupby(by='base_asset').agg('sum')
            importance_weighted_bid_price = importance_weighted_bid_price['importance_weighted_bid_price']
            conversion_table['traded_bid_price'] = \
                conversion_table.apply(lambda x: importance_weighted_bid_price.loc[x['base_asset']], axis='columns')
            importance_weighted_ask_price = conversion_table.groupby(by='base_asset').agg('sum')
            importance_weighted_ask_price = importance_weighted_ask_price['importance_weighted_ask_price']
            conversion_table['traded_ask_price'] = \
                conversion_table.apply(lambda x: importance_weighted_ask_price.loc[x['base_asset']], axis='columns')

            conversion_table['traded_bid_ask_percent_change'] = \
                ((conversion_table['traded_ask_price'] - conversion_table['traded_bid_price']) / \
                 conversion_table['traded_ask_price'])
            conversion_table['traded_bid_ask_volume_percent_change'] = \
                (conversion_table['traded_bid_volume'] / (conversion_table['traded_bid_volume'] + \
                                                          conversion_table['traded_ask_volume']))
            conversion_table[['traded_bid_ask_percent_change', 'traded_bid_ask_volume_percent_change']] *= 100

        conversion_table = conversion_table[~conversion_table['is_shorted']]

    conversion_table['bid_ask_percent_change'] = \
        ((conversion_table['ask_price'] - conversion_table['bid_price']) / conversion_table['ask_price'])
    conversion_table['bid_ask_volume_percent_change'] = \
        (conversion_table['bid_volume'] / (conversion_table['bid_volume'] + conversion_table['ask_volume']))
    conversion_table[['bid_ask_percent_change', 'bid_ask_volume_percent_change']] *= 100
    if as_pair and convert_to_USDT:
        if minimal:
            if extra_minimal:
                conversion_table = \
                    conversion_table[['symbol', 'base_asset', 'quote_asset', 'price_change_percent', 'close', 
                                      'bid_price', 'bid_volume', 'ask_price', 'ask_volume', 'close_time', 
                                      'count', 'rolling_base_volume', 'rolling_quote_volume', 
                                      'USDT_price_change_percent', 'USDT_price', 'rolling_USDT_base_volume', 
                                      'rolling_USDT_quote_volume', 'rolling_traded_volume', 'traded_price', 
                                      'bid_ask_percent_change', 'bid_ask_volume_percent_change']]
            else:
                conversion_table = \
                    conversion_table[['symbol', 'base_asset', 'quote_asset', 'price_change_percent', 'close', 
                                      'bid_price', 'bid_volume', 'ask_price', 'ask_volume', 'close_time', 
                                      'count', 'rolling_base_volume', 'rolling_quote_volume', 
                                      'USDT_price_change_percent', 'USDT_price', 'rolling_USDT_base_volume', 
                                      'rolling_USDT_quote_volume', 'USDT_bid_price', 'USDT_ask_price', 
                                      'USDT_bid_volume', 'USDT_ask_volume', 'rolling_traded_volume', 
                                      'traded_bid_volume', 'traded_ask_volume', 'traded_price', 
                                      'traded_bid_price', 'traded_ask_price', 'bid_ask_percent_change', 
                                      'bid_ask_volume_percent_change', 'traded_bid_ask_percent_change', 
                                      'traded_bid_ask_volume_percent_change']]
        else:
            conversion_table = \
                conversion_table[['symbol', 'base_asset', 'quote_asset', 'is_shorted', 'price_change_percent', 
                                  'weighted_average_price', 'open', 'high', 'low', 'close', 'close_shifted', 
                                  'last_volume', 'bid_price', 'bid_volume', 'ask_price', 'ask_volume', 
                                  'close_time', 'last_ID', 'count', 'rolling_base_volume', 'rolling_quote_volume', 
                                  'importance', 'USDT_price_change_percent', 'USDT_open', 
                                  'USDT_high', 'USDT_low', 'USDT_price', 'rolling_USDT_base_volume', 
                                  'rolling_USDT_quote_volume', 'USDT_bid_price', 'USDT_ask_price', 
                                  'USDT_bid_volume', 'USDT_ask_volume', 'rolling_traded_volume', 
                                  'traded_bid_volume', 'traded_ask_volume', 'traded_price', 
                                  'traded_bid_price', 'traded_ask_price', 'bid_ask_percent_change', 
                                  'bid_ask_volume_percent_change', 'traded_bid_ask_percent_change', 
                                  'traded_bid_ask_volume_percent_change']]
    else:
        if convert_to_USDT:
            if minimal:
                if extra_minimal:
                    if super_extra_minimal:
                        conversion_table = \
                            conversion_table[['base_asset', 'price_change_percent', 'close_time', 
                                              'bid_volume', 'ask_volume', 'bid_price', 'close', 'ask_price', 
                                              'count', 'rolling_traded_volume', 'bid_ask_percent_change', 
                                              'bid_ask_volume_percent_change', 'traded_price']]
                    else:
                        conversion_table = \
                            conversion_table[['base_asset', 'USDT_price_change_percent', 'close_time', 
                                              'bid_volume', 'ask_volume', 'bid_price', 'close', 'ask_price', 
                                              'count', 'rolling_traded_volume', 'bid_ask_percent_change', 
                                              'bid_ask_volume_percent_change', 'traded_price']]
                else:
                    conversion_table = \
                        conversion_table[['base_asset', 'USDT_price_change_percent', 'close_time', 'count', 
                                          'rolling_traded_volume', 'traded_bid_volume', 
                                          'traded_ask_volume', 'traded_price', 'traded_bid_price', 
                                          'traded_ask_price', 'traded_bid_ask_percent_change', 
                                          'traded_bid_ask_volume_percent_change']]
            else:
                conversion_table = \
                    conversion_table[['base_asset', 'USDT_price_change_percent', 'close_time', 'last_ID', 
                                      'count', 'rolling_traded_volume', 'traded_bid_volume', 
                                      'traded_ask_volume', 'traded_price', 'traded_bid_price', 
                                      'traded_ask_price', 'traded_bid_ask_percent_change', 
                                      'traded_bid_ask_volume_percent_change']]
            conversion_table['rolling_quote_volume'] = conversion_table['rolling_traded_volume'].copy()
            if 'close' in conversion_table.columns:
                conversion_table = conversion_table.drop(columns=['close'])
            if extra_minimal:
                conversion_table = \
                    conversion_table.rename(columns={'rolling_traded_volume': 'rolling_base_volume', 
                                                     'traded_price': 'close'})
                if not super_extra_minimal:
                    conversion_table = \
                        conversion_table.rename(columns={'USDT_price_change_percent': 'price_change_percent'})
            else:
                conversion_table = \
                    conversion_table.rename(columns={'USDT_price_change_percent': 'price_change_percent', 
                                                     'rolling_traded_volume': 'rolling_base_volume', 
                                                     'traded_bid_volume': 'bid_volume', 
                                                     'traded_ask_volume': 'ask_volume', 
                                                     'traded_price': 'close', 
                                                     'traded_bid_price': 'bid_price', 
                                                     'traded_ask_price': 'ask_price', 
                                                     'traded_bid_ask_percent_change': 'bid_ask_percent_change', 
                                                     'traded_bid_ask_volume_percent_change': 
                                                             'bid_ask_volume_percent_change'})
        if not as_pair:
            conversion_table['symbol'] = conversion_table['base_asset'].copy()
            conversion_table['quote_asset'] = conversion_table['base_asset'].copy()
            if minimal:
                df = conversion_table.groupby(by=['base_asset']).agg({'close_time': 'max', 'count': 'sum'})
                conversion_table.loc[:, ['close_time', 'count']] = \
                    conversion_table.apply(lambda x: df.loc[x['base_asset']], axis='columns')
                if extra_minimal:
                    df = conversion_table.groupby(by=['base_asset']).agg({'bid_ask_percent_change': 'min', 
                                                                          'bid_ask_volume_percent_change': 'max'})
                    conversion_table.loc[:, ['bid_ask_percent_change', 'bid_ask_volume_percent_change']] = \
                        conversion_table.apply(lambda x: df.loc[x['base_asset']], axis='columns')
            else:
                df = conversion_table.groupby(by=['base_asset']).agg({'close_time': 'max', 
                                                                      'last_ID': 'sum', 
                                                                      'count': 'sum'})
                conversion_table.loc[:, ['close_time', 'last_ID', 'count']] = \
                    conversion_table.apply(lambda x: df.loc[x['base_asset']], axis='columns')
            conversion_table = conversion_table.drop_duplicates(subset=['base_asset'], keep='first')
        conversion_table = conversion_table.reset_index(drop=True)
    return conversion_table.set_index('close_time').sort_index()

def get_conversion_table(client, exchange_info, offset_s=0, dump_raw=False, as_pair=True, minimal=False, 
                         extra_minimal=False, super_extra_minimal=False, convert_to_USDT=False):
    conversion_table = get_conversion_table_from_binance(client=client, exchange_info=exchange_info, 
                                                         offset_s=offset_s, dump_raw=dump_raw)
    return process_conversion_table(conversion_table=conversion_table, exchange_info=exchange_info, 
                                    as_pair=as_pair, minimal=minimal, extra_minimal=extra_minimal, 
                                    super_extra_minimal=super_extra_minimal, convert_to_USDT=convert_to_USDT)

In [21]:
#from cryptocurrency.conversion_table import get_conversion_table
import time

t1 = time.time()
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, 
                                        offset_s=offset_s, dump_raw=False, as_pair=False, 
                                        minimal=True, extra_minimal=True, 
                                        super_extra_minimal=True, convert_to_USDT=True)
t2 = time.time()
print('Elapsed time:', t2 - t1)
conversion_table.sort_values(by='rolling_base_volume', ascending=False).head(50)

Elapsed time: 8.946098327636719


,base_asset,price_change_percent,bid_volume,ask_volume,bid_price,ask_price,count,rolling_base_volume,bid_ask_percent_change,bid_ask_volume_percent_change,close,rolling_quote_volume,symbol,quote_asset
close_time,,,,,,,,,,,,,,
2022-12-09 23:33:46.578,BTC,-0.885,2.491000e-02,8.227000e-02,2.681708e+08,2.684000e+08,6900027,7.193717e+09,0.000058,96.934558,16667.510193,7.193717e+09,BTC,BTC
2022-12-09 23:33:41.909,USDT,-1.300,5.333000e+02,5.439000e+02,7.440000e+02,7.446000e+02,105144,6.734521e+09,0.009999,97.470468,288.558389,6.734521e+09,USDT,USDT
2022-12-09 23:33:46.686,BUSD,1.208,9.427000e+02,4.971500e+03,1.031000e+00,1.034000e+00,370531,4.380301e+09,0.009999,99.745144,218.858106,4.380301e+09,BUSD,BUSD
2022-12-09 23:33:46.665,ETH,-1.594,2.082000e-01,3.034600e+00,2.221200e+04,2.225100e+04,912444,9.274182e+08,0.000791,98.434329,1229.952780,9.274182e+08,ETH,ETH
2022-12-09 23:33:46.379,CTXC,67.175,7.000000e+02,1.070000e+03,2.071000e-01,2.074000e-01,946861,1.927264e+08,0.096525,94.611771,0.207100,1.927264e+08,CTXC,CTXC
2022-12-09 23:33:46.153,BNB,-1.743,1.600000e-02,1.100000e-01,1.121800e+04,1.124700e+04,314674,1.797675e+08,0.005989,84.742976,239.039763,1.797675e+08,BNB,BNB
2022-12-09 23:33:46.513,XRP,-1.701,5.140000e+02,7.700000e+02,2.439000e+01,2.444000e+01,212896,1.556846e+08,0.025667,92.172857,0.408802,1.556846e+08,XRP,XRP
2022-12-09 23:33:46.111,DOGE,-2.086,2.071000e+03,2.062000e+03,6.057000e+00,6.083000e+00,268455,1.308095e+08,0.010323,95.402405,131.740216,1.308095e+08,DOGE,DOGE
2022-12-09 23:33:46.295,FET,37.459,2.000000e+02,1.352000e+03,4.210000e-04,4.230000e-04,592442,1.202975e+08,0.082850,12.886598,0.120700,1.202975e+08,FET,FET


In [ ]:
conversion_table['quote_asset'].unique()

In [ ]:
def get_connected_assets(asset, exchange_info, priority='accuracy'):
    def reorder(connected_assets, priority):
        prioritized = [asset for asset in priority if asset in connected_assets]
        order = {asset: i for i, asset in enumerate(prioritized)}
        connected_assets_items = [asset for asset in connected_assets if asset in order]
        connected_assets_items.sort(key=order.get)
        connected_assets_iter = iter(connected_assets_items)
        return [next(connected_assets_iter) if asset in order 
                else asset for asset in connected_assets]
    priorities = {}
    priorities['accuracy'] = ['USDT', 'BTC', 'BUSD', 'ETH', 'BNB', 'AUD']
    priorities['fees'] = ['BUSD', 'BTC', 'BNB', 'ETH', 'USDT', 'AUD']
    priorities['wallet'] = ['BTC', 'ETH', 'BUSD', 'BNB', 'USDT', 'AUD']
    priority = priorities[priority]
    connected_base_assets = exchange_info['base_asset'] == asset
    connected_base_assets = exchange_info[connected_base_assets]
    connected_base_assets = connected_base_assets['quote_asset'].tolist()
    connected_quote_assets = exchange_info['quote_asset'] == asset
    connected_quote_assets = exchange_info[connected_quote_assets]
    connected_quote_assets = connected_quote_assets['base_asset'].tolist()
    connected_assets = sorted(list(set(connected_base_assets + connected_quote_assets)))
    prioritized = reorder(connected_assets, priority)
    return prioritized

asset = 'AXS'
connected_assets = get_connected_assets(asset, exchange_info, priority='fees')
print(connected_assets)

In [ ]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion_table import get_conversion_table
import pandas as pd

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info

crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)

conversion_table = df_1min.copy()
conversion_table.columns = conversion_table.columns.swaplevel(0, 1)
conversion_table = conversion_table.drop(columns=['rolling_base_volume', 
                                                  'rolling_quote_volume'])
conversion_table.columns = conversion_table.columns.swaplevel(0, 1)
conversion_table

In [ ]:
from cryptocurrency.conversion import convert_ohlcvs_from_pairs_to_assets

new_conversion_table = convert_ohlcvs_from_pairs_to_assets(conversion_table, 
                                                           exchange_info)
new_conversion_table

In [ ]:
new_conversion_table['ETH']